QUICK NOTES: convert month, day, hour, etc. to object dtype??

 # This notebook is to join both datasets: Taxis and Weather
 The Taxis dataset should have recorded data for each hourly period for each Manhattan zone (LocationID). This is not the case, a few LocationIDs are missing in some hourly periods.<br>
Therefore I will create a completed LocationID data frame and perform a left JOIN with the Taxis dataset:
1. First, I need to **fill the missing LocationIDs in the Taxis dataset.**<br>
1.1. Create Manhatan Zones DataFrame (LocationIDs)<br>
1.2. Create Data Frame index with LocationIDs<br>
1.3. Import cleaned Taxis Dataset & Sanity check<br>
1.4. Check that both Multi indexes are the same (for the Join)<br>
1.5. Create Multi Index with groupby & Sanity check<br>
1.6. Perform the JOIN & Sanity Check<br>


2. Second, I will **join Taxis and Weather Dataset**.<br>
2.1. Import cleaned Weather dataset & Sanity check.<br>
2.2. Insert Datetime column in Taxis dataset.<br>
2.3. Merge Taxis and Weather datasets.<br>
2.4. Manage NaNs.<br>
2.5. Save to CSV.

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime as dt
import matplotlib.pyplot as plt
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
# Let´s start by joining just one year: 2017

## 1. Fill missing LocationIDs in the Taxis dataset

### 1.1. Create Manhatan Zones DataFrame (LocationIDs)

- There should be 67 unique LocationIDs.
- The DataFrame should have 586.920 rows:
    - 365 days * 24h * 67 LocationIDs = 586.920

In [2]:
# 1. Import Location and Borough columns form NY TAXI ZONES dataset
dfzones = pd.read_csv('../data/NY_taxi_zones.csv', sep=',',
                      usecols=['LocationID', 'borough'])

# 2. Filter Manhattan zones
dfzones = dfzones[dfzones['borough']=='Manhattan']\
                .drop(['borough'], axis=1)\
                .sort_values(by='LocationID')\
                .drop_duplicates('LocationID').reset_index(drop=True)

#dfzones.PULocationID = dfzones.PULocationID.astype(str)

dfzones = pd.concat([dfzones]*8760).reset_index(drop=True)

print(dfzones.shape)
print('67 UNIQUE LocationIDs',pd.unique(dfzones['LocationID']).shape)

dfzones.head()

#dfManZon_array = dfManZon.iloc[:,0].values
#dfManZon_array
# There are 67 zones in Manhattan

(586920, 1)
67 UNIQUE LocationIDs (67,)


,LocationID
0,4
1,12
2,13
3,24
4,41


### 1.2. Create Data Frame index with LocationIDs
This Multi Index is necessary to perform the Join successfully.

In [3]:
a = pd.period_range(start='2017-01-01', end='2017-12-31T23:00', freq='H')
df_index = pd.DataFrame({'datetime':a})

df_index['month'] = df_index['datetime'].dt.month
df_index['day'] = df_index['datetime'].dt.day
df_index['hour'] = df_index['datetime'].dt.hour
df_index = df_index.drop(columns=['datetime'],inplace=False)
df_index = df_index.iloc[np.arange(len(df_index)).repeat(67)].reset_index(drop=True)
df_index['LocationID'] = dfzones['LocationID']
print(df_index.shape)
df_index.head()

(586920, 4)


,month,day,hour,LocationID
0,1,1,0,4
1,1,1,0,12
2,1,1,0,13
3,1,1,0,24
4,1,1,0,41


### 1.3. Import cleaned Taxis Dataset & Sanity check
- Confirm that the year is correct.
- Confirm that hourly periods count is correct

In [21]:
year = 2017
dftax = pd.read_csv('../data/Data_Taxis_'+str(year)+'_Cleaned.csv', sep=',',
                        #dtype = {"PULocationID" : "object"},
                        parse_dates={'datetime':['pickup_datetime']})
print('Year should be unique: ', dftax.year.unique())

print('67 UNIQUE LocationIDs: ', pd.unique(dftax['LocationID']).shape)
print(dftax.shape[0], 'A number less than 586920 indicates that there are missing LocationIDs')

# Count LocationID per hourly period. They should be 67.
t = dftax.copy()
t['count'] = 1
tg = t.groupby(['month', 'day', 'hour']).sum().head()
print('Count unique LocationIDs per hourly period. Should be 67: ',pd.unique(tg['count']))

display(tg.head())
display(dftax.head())

Year should be unique:  [2017]
67 UNIQUE LocationIDs:  (67,)
536306 A number less than 586920 indicates that there are missing LocationIDs
Count unique LocationIDs per hourly period. Should be 67:  [65 64]


LocationID  NoOfPickups    year  week  dayofweek  isweekend  \
month day hour                                                                
1     1   0           9733        18881  131105  3380        390         65   
          1           9539        20186  129088  3328        384         64   
          2           9667        17989  131105  3380        390         65   
          3           9667        14997  131105  3380        390         65   
          4           9547        10700  129088  3328        384         64   

                IsHoliday  count  
month day hour                    
1     1   0             0     65  
          1             0     64  
          2             0     65  
          3             0     65  
          4             0     64

,datetime,LocationID,NoOfPickups,year,month,day,hour,week,dayofweek,isweekend,IsHoliday
0,2017-01-01,4,136,2017,1,1,0,52,6,1,0
1,2017-01-01,12,3,2017,1,1,0,52,6,1,0
2,2017-01-01,13,103,2017,1,1,0,52,6,1,0
3,2017-01-01,24,94,2017,1,1,0,52,6,1,0
4,2017-01-01,41,136,2017,1,1,0,52,6,1,0


### 1.4. Check that both Multi indexes are the same (for the Join)
After many "trial & error" I have come to the conclusion that I need to create a Multi Index based on 'month', 'day', 'hour' and 'LocationID' in order to perform the JOIN successfully.<br>
I will check that the parameters needed for the multi index are correct.

In [5]:
tax_m = dftax.groupby(['month']).count()
tax_d = dftax.groupby(['month', 'day']).count()
tax_h = dftax.groupby(['month', 'day','hour']).count()
ind_m = df_index.groupby(['month']).count()
ind_d = df_index.groupby(['month','day']).count()
ind_h = df_index.groupby(['month','day','hour']).count()
ind_z = df_index.groupby(['month','day','hour','LocationID']).count()


print('12 MONTHS:',tax_m.shape[0],'=>',ind_m.shape[0])
print('365 DAYS:',tax_d.shape[0],'=>', ind_d.shape[0])
print('8760 HOURS:',tax_h.shape[0],'=>', ind_h.shape[0])
print('67 UNIQUE LocationID:',\
      pd.unique(df_index['LocationID']).shape[0],'=>',\
      pd.unique(dftax['LocationID']).shape[0])

12 MONTHS: 12 => 12
365 DAYS: 365 => 365
8760 HOURS: 8760 => 8760
67 UNIQUE LocationID: 67 => 67


### 1.5. Create Multi Index with groupby & Sanity check

In [6]:
dftax_g = dftax.groupby(['month','day','hour','LocationID']).sum()
df_index_g = df_index.groupby(['month','day','hour','LocationID']).sum()
print(dftax_g.shape)
print(df_index_g.shape)

(536306, 6)
(586920, 0)


In [7]:
# Sanity check
print('Taxis dataset BEFORE grouping.')
print(dftax.shape[0])
print('67 UNIQUE LocationIDs: ', pd.unique(dftax['LocationID']).shape[0])
display(dftax.head())

print('Taxis dataset AFTER grouping.')
print('It should be 586920: ', df_index_g.shape[0])
print('67 UNIQUE LocationIDs: ',dftax_g.index.unique(level='LocationID').shape[0])
display(dftax_g.head(100))

Taxis dataset BEFORE grouping.
536306
67 UNIQUE LocationIDs:  67


,datetime,LocationID,NoOfPickups,year,month,day,hour,week,dayofweek,isweekend,IsHoliday
0,2017-01-01,4,136,2017,1,1,0,52,6,1,0
1,2017-01-01,12,3,2017,1,1,0,52,6,1,0
2,2017-01-01,13,103,2017,1,1,0,52,6,1,0
3,2017-01-01,24,94,2017,1,1,0,52,6,1,0
4,2017-01-01,41,136,2017,1,1,0,52,6,1,0


Taxis dataset AFTER grouping.
It should be 586920:  586920
67 UNIQUE LocationIDs:  67


NoOfPickups  year  week  dayofweek  isweekend  \
month day hour LocationID                                                  
1     1   0    4                   136  2017    52          6          1   
               12                    3  2017    52          6          1   
               13                  103  2017    52          6          1   
               24                   94  2017    52          6          1   
               41                  136  2017    52          6          1   
               42                   79  2017    52          6          1   
               43                  401  2017    52          6          1   
               45                   54  2017    52          6          1   
               48                  692  2017    52          6          1   
               50                  313  2017    52          6          1   
               68                  659  2017    52          6          1   
               74                  124  2017    52          6          1   
               75                  147  2017    52          6          1   
               79                 1070  2017    52          6          1   
               87                  150  2017    52          6          1   
               88                   70  2017    52          6          1   
               90                  467  2017    52          6          1   
               100                  98  2017    52          6          1   
               107                 623  2017    52          6          1   
               113                 317  2017    52          6          1   
               114                 406  2017    52          6          1   
               116                  69  2017    52          6          1   
               120                   1  2017    52          6          1   
               125                  98  2017    52          6          1   
               127                  13  2017    52          6          1   
               137                 302  2017    52          6          1   
               140                 229  2017    52          6          1   
               141                 522  2017    52          6          1   
               142                 735  2017    52          6          1   
               143                 234  2017    52          6          1   
               144                 269  2017    52          6          1   
               148                 588  2017    52          6          1   
               151                 238  2017    52          6          1   
               152                  39  2017    52          6          1   
               153                   1  2017    52          6          1   
               158                 297  2017    52          6          1   
               161                 640  2017    52          6          1   
               162                 415  2017    52          6          1   
               163                 222  2017    52          6          1   
               164                 476  2017    52          6          1   
               166                 108  2017    52          6          1   
               170                 670  2017    52          6          1   
               186                 335  2017    52          6          1   
               194                   1  2017    52          6          1   
               202                   2  2017    52          6          1   
               209                  49  2017    52          6          1   
               211                 184  2017    52          6          1   
               224                  61  2017    52          6          1   
               229                 520  2017    52          6          1   
               230                  30  2017    52          6          1   
               231                 343  2017    52          6          1   

### 1.6. Perform the JOIN & Sanity Check
This Join will fill the missing LocationIDs within the taxis dataset.

In [66]:
taxis_join = dftax_g.join(df_index_g, how='right').reset_index()

In [67]:
# Sanity Check
print('67 Unique LocationIDs in total: ',pd.unique(taxis_join['LocationID']).shape)
print('Shape should be (586920, x): ', taxis_join.shape)
t2 = taxis_d_final.copy()
t2['count'] = 1
tg2 = t2.groupby(['month', 'day', 'hour']).sum().head()
print('Count unique LocationIDs per hourly period. Should be 67: ',pd.unique(tg2['count']))
display(tg2.head())

display(taxis_join.head())

67 Unique LocationIDs in total:  (67,)
Shape should be (586920, x):  (586920, 10)
Count unique LocationIDs per hourly period. Should be 67:  [67]


LocationID  NoOfPickups      year    week  dayofweek  \
month day hour                                                         
1     1   0           9964      18881.0  131105.0  3380.0      390.0   
          1           9964      20186.0  129088.0  3328.0      384.0   
          2           9964      17989.0  131105.0  3380.0      390.0   
          3           9964      14997.0  131105.0  3380.0      390.0   
          4           9964      10700.0  129088.0  3328.0      384.0   

                isweekend  IsHoliday  count  
month day hour                               
1     1   0          65.0        0.0     67  
          1          64.0        0.0     67  
          2          65.0        0.0     67  
          3          65.0        0.0     67  
          4          64.0        0.0     67

,month,day,hour,LocationID,NoOfPickups,year,week,dayofweek,isweekend,IsHoliday
0,1,1,0,4,136.0,2017.0,52.0,6.0,1.0,0.0
1,1,1,0,12,3.0,2017.0,52.0,6.0,1.0,0.0
2,1,1,0,13,103.0,2017.0,52.0,6.0,1.0,0.0
3,1,1,0,24,94.0,2017.0,52.0,6.0,1.0,0.0
4,1,1,0,41,136.0,2017.0,52.0,6.0,1.0,0.0


## 2. Join Taxis and Weather datasets

### 2.1. Import cleaned Weather dataset & Sanity check.
Only one year (for now)

In [ ]:
# Filter one year data

In [15]:
# Import WEATHER DATASET to dataframe.
dfwea = pd.read_csv('../data/Data_Weather_Cleaned.csv', sep=',',
                        parse_dates={'datetime':['DATE']})

# Filter one year data
dfwea.drop(dfwea[dfwea['datetime'] < pd.Timestamp(date(year,1,1))].index, inplace=True)
dfwea.drop(dfwea[dfwea['datetime'] >= pd.Timestamp(date(year+1,1,1))].index, inplace=True)

# Sanity check
print('Year should be unique: ', dfwea.datetime.dt.year.unique())
print('There should be 8760 hourly periods in a year: ', dfwea.shape[0])

dfwea.sample(5)

Year should be unique:  [2017]
There should be 8760 hourly periods in a year:  8760


,datetime,HourlyPrecipitation
7791,2017-11-21 15:00:00,0.0
8475,2017-12-20 03:00:00,0.0
3433,2017-05-24 01:00:00,0.0
4527,2017-07-08 15:00:00,0.0
7818,2017-11-22 18:00:00,0.0


### 2.2. Insert Datetime column in Taxis dataset.
I lost this column when creating the multi index so I need to put it back in order to perform the MERGE on 'datetime'.

In [74]:
# I will take the 'datetime' sequence from the Weather data frame
datetime_col = dfwea.copy()
datetime_col.drop(columns=['HourlyPrecipitation'], inplace=True)

# repeat values to have one hour per LocationID (67)
datetime_col = pd.DataFrame(np.repeat(datetime_col.values,67))
# rename column
datetime_col = datetime_col.rename(columns={0:'datetime'})

taxis_final = pd.concat([datetime_col,taxis_join], axis=1)
print('Should have 586920 rows: ',taxis_final.shape[0])
taxis_final.head()

Should have 586920 rows:  586920


,datetime,month,day,hour,LocationID,NoOfPickups,year,week,dayofweek,isweekend,IsHoliday
0,2017-01-01,1,1,0,4,136.0,2017.0,52.0,6.0,1.0,0.0
1,2017-01-01,1,1,0,12,3.0,2017.0,52.0,6.0,1.0,0.0
2,2017-01-01,1,1,0,13,103.0,2017.0,52.0,6.0,1.0,0.0
3,2017-01-01,1,1,0,24,94.0,2017.0,52.0,6.0,1.0,0.0
4,2017-01-01,1,1,0,41,136.0,2017.0,52.0,6.0,1.0,0.0


### 2.3. Merge Taxis and Weather datasets

In [87]:
df_merge = pd.merge(taxis_final, dfwea, on='datetime')
# taxis dataframe and merged dataframe should have same number of rows
print('df_merge shape ({0}) should be equal to taxis_final shape ({1})'.format(df_merge.shape[0], taxis_final.shape[0]))
df_merge.head(100)

df_merge shape (586920) should be equal to taxis_final shape (586920)


,datetime,month,day,hour,LocationID,NoOfPickups,year,week,dayofweek,isweekend,IsHoliday,HourlyPrecipitation
0,2017-01-01 00:00:00,1,1,0,4,136.0,2017.0,52.0,6.0,1.0,0.0,0.0
1,2017-01-01 00:00:00,1,1,0,12,3.0,2017.0,52.0,6.0,1.0,0.0,0.0
2,2017-01-01 00:00:00,1,1,0,13,103.0,2017.0,52.0,6.0,1.0,0.0,0.0
3,2017-01-01 00:00:00,1,1,0,24,94.0,2017.0,52.0,6.0,1.0,0.0,0.0
4,2017-01-01 00:00:00,1,1,0,41,136.0,2017.0,52.0,6.0,1.0,0.0,0.0
5,2017-01-01 00:00:00,1,1,0,42,79.0,2017.0,52.0,6.0,1.0,0.0,0.0
6,2017-01-01 00:00:00,1,1,0,43,401.0,2017.0,52.0,6.0,1.0,0.0,0.0
7,2017-01-01 00:00:00,1,1,0,45,54.0,2017.0,52.0,6.0,1.0,0.0,0.0
8,2017-01-01 00:00:00,1,1,0,48,692.0,2017.0,52.0,6.0,1.0,0.0,0.0
9,2017-01-01 00:00:00,1,1,0,50,313.0,2017.0,52.0,6.0,1.0,0.0,0.0


### 2.4. Manage NaNs
After filling the LocationID gaps, I need to manage the NaNs of the following variables:
    - NoOfPickups: will be 0.
    - year: will take calculate 'datetime'.
    - week: will take calculate 'datetime'.
    - dayofweek: will calculate from 'datetime'.
    - isweekend: will calculate from 'datetime'.
    - isholiday: will ignore for now.

In [90]:
df_merge['NoOfPickups'].fillna(0, inplace=True)
df_merge['year'] = df_merge['datetime'].dt.year
df_merge['week'] = df_merge['datetime'].dt.week
df_merge['dayofweek'] = df_merge['datetime'].dt.dayofweek
# isweekend
mask = (df_merge['dayofweek'] == 5) | (df_merge['dayofweek'] == 6)
df_merge['isweekend'] = np.where(mask, 1, 0)
# drop isholiday
df_merge.drop(columns=['IsHoliday'], inplace=True)
df_merge.head(20)

,datetime,month,day,hour,LocationID,NoOfPickups,year,week,dayofweek,isweekend,HourlyPrecipitation
0,2017-01-01,1,1,0,4,136.0,2017,52,6,1,0.0
1,2017-01-01,1,1,0,12,3.0,2017,52,6,1,0.0
2,2017-01-01,1,1,0,13,103.0,2017,52,6,1,0.0
3,2017-01-01,1,1,0,24,94.0,2017,52,6,1,0.0
4,2017-01-01,1,1,0,41,136.0,2017,52,6,1,0.0
5,2017-01-01,1,1,0,42,79.0,2017,52,6,1,0.0
6,2017-01-01,1,1,0,43,401.0,2017,52,6,1,0.0
7,2017-01-01,1,1,0,45,54.0,2017,52,6,1,0.0
8,2017-01-01,1,1,0,48,692.0,2017,52,6,1,0.0
9,2017-01-01,1,1,0,50,313.0,2017,52,6,1,0.0


### 2.5. Save to CSV

In [91]:
# save csv
df_merge.to_csv('../data/Data_Cleaned_'+str(year)+'_To_Model.csv', index = False, header=True)